Import thư viện

In [26]:
from cv2 import mean
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.preprocessing as pre


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

7 kĩ thuật xử lý dữ liệu trống

In [27]:
#Giá trị trung vị
def median():
    data  = pd.read_csv('titanic.csv')[['Survived', 'Age', 'Fare']]
    median=data['Age'].median()
    data['Age']=data['Age'].fillna(median)
    return data

In [28]:
#Giá trị trung bình
def mean():
    data  = pd.read_csv('titanic.csv')[['Survived', 'Age', 'Fare']]
    mean = data['Age'].mean()
    data['Age']=data['Age'].fillna(mean)
    return data

In [29]:
#Giá trị xuất hiện nhiều nhất
def mode():
    data  = pd.read_csv('titanic.csv')[['Survived', 'Age', 'Fare']]
    mode = data['Age'].mode()[0]
    data['Age']=data['Age'].fillna(mode)
    return data

In [30]:
#Giá trị được lấy ngẫu nhiên
def randomsample():
    data  = pd.read_csv('titanic.csv')[['Survived', 'Age', 'Fare']]
    random_samples = data['Age'].dropna().sample(n=data['Age'].isnull().sum(),random_state=0)
    # gan lai index cho series ngau nhien vua tao
    random_samples.index = data[data['Age'].isnull()].index
    data['Age'].update(random_samples)
    return data 

In [31]:
#Tạo đặc trưng mới
def new_feature():
    data  = pd.read_csv('titanic.csv')[['Survived', 'Age', 'Fare']]
    data['Age_NAN']=np.where(data['Age'].isnull(),1,0)
    data['Age'].fillna(data.Age.median(),inplace=True)
    return data

In [32]:
#Giá trị bất kỳ
def arbitrary_value():
    data  = pd.read_csv('titanic.csv')[['Survived', 'Age', 'Fare']]
    data['Age']=data['Age'].fillna(100)
    return data

In [33]:
#Giá trị tại đuôi của phân bố dữ liệu
def end_of_dist():
    data  = pd.read_csv('titanic.csv')[['Survived', 'Age', 'Fare']]
    # gia tri o duoi cua phan bo (bien Age theo phan bo chuan)
    extreme = data['Age'].mean() + 3*data['Age'].std()
    data["Age"]=data['Age'].fillna(extreme)
    return data

Không xử lý ngoại lệ và có xử lý ngoại lệ

In [34]:
def no_outliers(data):
    return logistic(data)

In [35]:
def outliers(data):
    #Dữ liệu dạng phân bố chuẩn
    upper_boundary=data['Age'].mean() + 3* data['Age'].std()
    lower_boundary=data['Age'].mean() - 3* data['Age'].std()
    
    #update các giá trị, nếu lớn hơn upper_boundary thì sẽ bằng giá trị upper_boundary
    data.loc[data['Age'] >= round(upper_boundary), 'Age'] = round(upper_boundary)

    #Dữ liệu dạng phân bố lệch
    #q3 tương ứng 75, q1 tương ứng 25
    q3 , q1 = np.percentile(data['Fare'], [75,25])
    IQR = q3 - q1
    upper_bridge = q3 + 3 * IQR
    lower_bridge = q1 - 3 * IQR

    #update các giá trị, nếu lớn hơn upper_bridge thì sẽ bằng giá trị upper_bridge
    data.loc[data['Fare'] >= upper_bridge, 'Fare'] = round(upper_bridge)

    return logistic(data)

Độ chính xác thuật toán của hồi quy Logistic

In [36]:
def logistic(data):
    accuracy_arr = []
    for i in range(10):
        X_train,X_test,y_train,y_test=train_test_split(data[['Age','Fare']],data['Survived'],test_size=0.3, random_state=i)
        classifier=LogisticRegression()
        classifier.fit(X_train,y_train)
        y_pred=classifier.predict(X_test)
        accuracy_arr.append(accuracy_score(y_test,y_pred))
    return np.mean(accuracy_arr)

Xử lý riêng cho phương pháp new feature

In [37]:
def no_outliers_for_new_feature(data):
    return logistic_for_new_feature(data)

def outlierss_for_new_feature(data):
    #Dữ liệu dạng phân bố chuẩn
    upper_boundary=data['Age'].mean() + 3* data['Age'].std()
    lower_boundary=data['Age'].mean() - 3* data['Age'].std()
    
    #update các giá trị, nếu lớn hơn upper_boundary thì sẽ bằng giá trị upper_boundary
    data.loc[data['Age'] >= round(upper_boundary), 'Age'] = round(upper_boundary)

    #Dữ liệu dạng phân bố lệch
    #q3 tương ứng 75, q1 tương ứng 25
    q3 , q1 = np.percentile(data['Fare'], [75,25])
    IQR = q3 - q1
    upper_bridge = q3 + 3 * IQR
    lower_bridge = q1 - 3 * IQR

    #update các giá trị, nếu lớn hơn upper_bridge thì sẽ bằng giá trị upper_bridge
    data.loc[data['Fare'] >= upper_bridge, 'Fare'] = round(upper_bridge)

    return logistic_for_new_feature(data)
    
def logistic_for_new_feature(data):
    accuracy_arr = []
    for i in range(10):
        X_train,X_test,y_train,y_test=train_test_split(data[['Age','Age_NAN','Fare']],data['Survived'],test_size=0.3, random_state=i)
        classifier=LogisticRegression()
        classifier.fit(X_train,y_train)
        y_pred=classifier.predict(X_test)
        accuracy_arr.append(accuracy_score(y_test,y_pred))
    return np.mean(accuracy_arr)

In [38]:
dataframe = {
        'Kỹ thuật xử lý dữ liệu trống': ['Mean', 'Median', 'Mode', 'Random', 'End of Distribution', 'Arbitrary Value', 'New Feature'],
        
        'Không xử lý ngoại lệ': 
        [
        no_outliers(mean()), 
        no_outliers(median()), 
        no_outliers(mode()), 
        no_outliers(randomsample()), 
        no_outliers(end_of_dist()), 
        no_outliers(arbitrary_value()), 
        no_outliers_for_new_feature(new_feature())
        ],

        'Có xử lý ngoại lệ':  
        [
        outliers(mean()), 
        outliers(median()), 
        outliers(mode()), 
        outliers(randomsample()), 
        outliers(end_of_dist()), 
        outliers(arbitrary_value()), 
        outlierss_for_new_feature(new_feature())
        ]}
statistics = pd.DataFrame(dataframe)

Báo cáo độ chính xác của 14 thuật toán trên

In [39]:
#Xuất ra 14 trường hợp
print("Bảng báo cáo độ chính xác của thuật toán trong 14 trường hợp\n\n",statistics)

Bảng báo cáo độ chính xác của thuật toán trong 14 trường hợp

   Kỹ thuật xử lý dữ liệu trống  Không xử lý ngoại lệ  Có xử lý ngoại lệ
0                         Mean              0.652985           0.667164
1                       Median              0.653358           0.667164
2                         Mode              0.653358           0.665299
3                       Random              0.654478           0.667537
4          End of Distribution              0.663806           0.666418
5              Arbitrary Value              0.669403           0.667537
6                  New Feature              0.660075           0.663433


KHÔNG XỬ LÝ NGOẠI LỆ

In [40]:
# Lớn nhất 
print("--------------------------------------------------------------------------")
print(statistics.loc[statistics['Không xử lý ngoại lệ']
.idxmax()][['Kỹ thuật xử lý dữ liệu trống', 'Không xử lý ngoại lệ']])
print("--------------------------------------------------------------------------")

#Nhỏ nhất
print(statistics.loc[statistics['Không xử lý ngoại lệ']
.idxmin()][['Kỹ thuật xử lý dữ liệu trống', 'Không xử lý ngoại lệ']])
print("--------------------------------------------------------------------------")

--------------------------------------------------------------------------
Kỹ thuật xử lý dữ liệu trống    Arbitrary Value
Không xử lý ngoại lệ                   0.669403
Name: 5, dtype: object
--------------------------------------------------------------------------
Kỹ thuật xử lý dữ liệu trống        Mean
Không xử lý ngoại lệ            0.652985
Name: 0, dtype: object
--------------------------------------------------------------------------


CÓ XỬ LÝ NGOẠI LỆ

In [41]:
# Lớn nhất 
print("--------------------------------------------------------------------------")
print(statistics.loc[statistics['Có xử lý ngoại lệ']
.idxmax()][['Kỹ thuật xử lý dữ liệu trống', 'Có xử lý ngoại lệ']])
print("--------------------------------------------------------------------------")

#Nhỏ nhất
print(statistics.loc[statistics['Không xử lý ngoại lệ']
.idxmin()][['Kỹ thuật xử lý dữ liệu trống', 'Có xử lý ngoại lệ']])
print("--------------------------------------------------------------------------")

--------------------------------------------------------------------------
Kỹ thuật xử lý dữ liệu trống      Random
Có xử lý ngoại lệ               0.667537
Name: 3, dtype: object
--------------------------------------------------------------------------
Kỹ thuật xử lý dữ liệu trống        Mean
Có xử lý ngoại lệ               0.667164
Name: 0, dtype: object
--------------------------------------------------------------------------


Đề xuất các kỹ thuật chuẩn hoá dữ liệu phù hợp để cải thiện độ chính xác của thuật toán hồi quy logistic

Robust Scaler

In [42]:
print("Kĩ thuật chuẩn hoá Robust Scaler: \n")

#chưa qua xử lý
accuracy_non_process = outliers(arbitrary_value())
print("Độ chính xác thuật toán không áp dụng kĩ thuật chuẩn hoá : ", accuracy_non_process)

#đã qua xử lý
arbitrary_value_copy = arbitrary_value()
arbitrary_value_copy = pd.DataFrame(pre.RobustScaler().fit_transform(arbitrary_value_copy), columns=arbitrary_value_copy.columns)
accuracy_process = outliers(arbitrary_value_copy)
print("\nĐộ chính xác thuật toán áp dụng kĩ thuật chuẩn hoá : ", accuracy_process)

#phân trăm khác biệt giữa 2 thuật toán
diffirent_percentage = round((accuracy_process - accuracy_non_process)*100,3)
print("\nPhần trăm khác biệt giữa 2 thuật toán : ", diffirent_percentage,"%")

Kĩ thuật chuẩn hoá Robust Scaler: 

Độ chính xác thuật toán không áp dụng kĩ thuật chuẩn hoá :  0.6675373134328357

Độ chính xác thuật toán áp dụng kĩ thuật chuẩn hoá:  0.6682835820895523

Phần trăm khác biệt giữa 2 thuật toán :  0.075 %


Min Max Scaler

In [43]:
print("Kĩ thuật chuẩn hoá Min Max Scaler: \n")

#chưa qua xử lý
accuracy_non_process = outliers(arbitrary_value())
print("Độ chính xác thuật toán không áp dụng kĩ thuật chuẩn hoá : ", accuracy_non_process)

#đã qua xử lý
arbitrary_value_copy = arbitrary_value()
arbitrary_value_copy = pd.DataFrame(pre.MinMaxScaler().fit_transform(arbitrary_value_copy), columns=arbitrary_value_copy.columns)
accuracy_process = outliers(arbitrary_value_copy)
print("\nĐộ chính xác thuật toán áp dụng kĩ thuật chuẩn hoá : ", accuracy_process)

#phân trăm khác biệt giữa 2 thuật toán
diffirent_percentage = round((accuracy_process - accuracy_non_process)*100,3)
print("\nPhần trăm khác biệt giữa 2 thuật toán : ", diffirent_percentage,"%")

Kĩ thuật chuẩn hoá Min Max Scaler: 

Độ chính xác thuật toán không áp dụng kĩ thuật chuẩn hoá :  0.6675373134328357

Độ chính xác thuật toán áp dụng kĩ thuật chuẩn hoá:  0.6164179104477613

Phần trăm khác biệt giữa 2 thuật toán :  -5.112 %
